## Data Transformation: Table Hospitals

### Datasets used:
- Hospitals bed capacity by state and county
- Hospital Charges in US

In [36]:
import pandas as pd

Load data to notebook

In [37]:
#Load data with pandas
bedCapacity = pd.read_csv("hospital_beds_USA.csv")
hospitalCharges = pd.read_csv("hospital-charges.csv")

Visualize data

In [38]:
bedCapacity.head(1)

,country,state,county,lat,lng,type,measure,beds,population,year,source,source_url
0,US,AK,aleutians east,63.588753,-154.493062,ICU,1000HAB,0.0,3338,2019,khn,https://khn.org/news/as-coronavirus-spreads-wi...


Cleaning data

In [39]:
#Cleaning Bed Capacity dataFrame
#Keeping data just for 2020
bedCapacity = (bedCapacity[bedCapacity.year == 2019]).reset_index() # there's no info available for all states in 2020
#Drop columns
bedCapacity = bedCapacity.drop(labels=(["country", "county", "lat", "lng", "measure", "source", "source_url", "index", "year"]), axis=1)
# group by state and type of bed
bedCapacity = bedCapacity.groupby(["state"]).sum().reset_index()


In [40]:
bedCapacity.head()

,state,beds,population
0,AK,4.789014,966872
1,AL,11.967593,4731663
2,AR,8.403947,2583665
3,AZ,2.050268,6630442
4,CA,8.182707,38982847


In [41]:
hospitalCharges.head(1)

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73


In [42]:
#Cleaning Hospital Charges dataFrame

#There's no year so we'll asume is data for 2020
#Drop columns
hospitalCharges = hospitalCharges.drop(labels=(["DRG Definition", "Provider Id", "Provider Street Address", "Provider City", 
                                        "Provider Zip Code", "Hospital Referral Region Description", "Average Medicare Payments", 
                                        "Provider Name"]), axis=1)
#Rename columns
hospitalCharges.columns = ['state', "totalDischarges", "averageCoveredCharges", "averageTotalPayments"]

In [43]:
hospitalCharges.head()

,state,totalDischarges,averageCoveredCharges,averageTotalPayments
0,AL,91,$32963.07,$5777.24
1,AL,14,$15131.85,$5787.57
2,AL,24,$37560.37,$5434.95
3,AL,25,$13998.28,$5417.56
4,AL,18,$31633.27,$5658.33


In [44]:
hospitalCharges["averageCoveredCharges"] = hospitalCharges["averageCoveredCharges"].str.replace("$","").astype("float")
hospitalCharges["averageTotalPayments"] = hospitalCharges["averageTotalPayments"].str.replace("$","").astype("float")
hospitalCharges = hospitalCharges.groupby("state").mean().reset_index()

/var/folders/8h/nd9bcmz12yndbs6hsp0s9w8r0000gn/T/ipykernel_16146/3456981773.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  hospitalCharges["averageCoveredCharges"] = hospitalCharges["averageCoveredCharges"].str.replace("$","").astype("float")
/var/folders/8h/nd9bcmz12yndbs6hsp0s9w8r0000gn/T/ipykernel_16146/3456981773.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  hospitalCharges["averageTotalPayments"] = hospitalCharges["averageTotalPayments"].str.replace("$","").astype("float")


Merge to get the final table

In [45]:
hospitals = bedCapacity.join(hospitalCharges.set_index("state"), on="state")
# hospitals.to_csv("hospitals.csv", sep=',') # generate csv

In [47]:
hospitals.head()

,state,beds,population,totalDischarges,averageCoveredCharges,averageTotalPayments
0,AK,4.789014,966872,26.588745,40348.743333,14572.391732
1,AL,11.967593,4731663,39.258322,31316.462074,7568.232149
2,AR,8.403947,2583665,41.978229,26174.526246,8019.248805
3,AZ,2.050268,6630442,36.690284,41200.063020,10154.528211
4,CA,8.182707,38982847,36.357854,67508.616536,12629.668472


Generate csv output

In [48]:
hospitals.to_csv("hospitals.csv", sep=',')